class DataCollector:
    def __init__(self):
        self.LOCAL_TZ = 'Asia/Almaty'
        self.SERVER_TZ = 'UTC'
        self.DATA_PATH='data/'
        self.ELO_DATA_PATH='data/elo/'
        self.PREREQUISITES_PATH='prerequisites/'
        self.COL_CAT=[]
        self.COL_NUM=[]
        self.COL_LBL=[]
        self.COL_INF=[]
    
    def _load_prerequisites(self,name):
        with open(os.path.join(self.PREREQUISITES_PATH, name),'rb') as f:
            encoder = pickle.load(f)
        return encoder
    
    def _save_prerequisite(self, name, data):
        os.makedirs(self.PREREQUISITES_PATH, mode=0o777, exist_ok=True)
        with open(os.path.join(self.PREREQUISITES_PATH, name), mode='wb') as f:
            pickle.dump(data, f) 

    def _ff(self, columns):
        if len(self.INCLUDE)>0:
            return [x for x in columns if x in self.INCLUDE]
        else:
            return [x for x in columns if x not in self.EXCLUDE]
    
    def _encode(self, enctype, features, outs, df):
        if (len(self.INCLUDE)>0 and outs[0] in self.INCLUDE) or outs[0] in self.EXCLUDE:
            return df
        name='_'.join(features)
        if self.LOAD:
            encoder=self._load_prerequisites(f'{enctype}_{features[0]}')
        else:
            if enctype=='sc':
                encoder = MinMaxScaler()
            elif enctype=='le':
                encoder = LabelEncoder()
            elif enctype=='ohe':
                encoder = OneHotEncoder()
            if len(features)==1:
                encoder.fit(df[features].values)
            else:
                encoder.fit(pd.concat([pd.DataFrame(df[features[0]].unique(), columns=[name]),pd.DataFrame(df[features[1]].unique(), columns=[name])])[name])
            self._save_prerequisite(f'{enctype}_{name}', encoder)
        if  enctype=='ohe':
            return encoder.transform(df[features].values).toarray()
        if len(features)==1:
            df[outs[0]] = encoder.transform(df[features].values)
        else:
            df[outs[0]] = encoder.transform(df[features[0]])
            df[outs[1]] = encoder.transform(df[features[1]])
        return df

    def _encode_teams(self, df):
        teams_name=self.ELO_DATA_PATH+'teams.csv'
        teams_saved=pd.read_csv(teams_name, index_col=None)
        teams=df[['team']].dropna().drop_duplicates()
        teams_new=teams[~teams.team.isin(teams_saved.team)]
        print(teams_new)
        if not teams_new.empty:
            print('New teams!')
            id=teams_saved.id.max()+1
            #id=0
            teams_list=[]
            for row in teams_new.itertuples():
                if len(row.team)>1:
                    teams_list.append({'team':row.team, 'id':id})
                    id+=1
                    #break
            teams_saved=pd.concat([teams_saved,pd.DataFrame(teams_list)])
            teams_saved.id=teams_saved.id.astype(int)
            teams_saved.to_csv(teams_name, index=False)
        df=df.merge(teams_saved, on='team', how='left')
        return df
    
    def _add_elo(self, df_src,df_elo):
        df_teams=pd.read_csv(self.DATA_PATH+'teams.csv', index_col=None)
        df_elo_merged=df_elo.merge(df_teams[['id','tid']], on='id', how='left').drop_duplicates()
        df_elo_merged=df_elo_merged.dropna()
        df_src['de']=df_src.ds.apply(lambda x: x.strftime('%Y-%m-%d'))
        df_elo_merged=df_elo_merged.rename(columns={'tid':'tid1', 'elo':'elo1'})
        df_src=df_src.merge(df_elo_merged[['tid1','de','elo1']], on=['tid1','de'], how='left')
        df_elo_merged=df_elo_merged.rename(columns={'tid1':'tid2', 'elo1':'elo2'})
        df_src=df_src.merge(df_elo_merged[['tid2','de','elo2']], on=['tid2','de'], how='left')
        return df_src

    def _provide_elo(self):
        df = pd.concat(map(pd.read_csv, glob.glob(os.path.join(self.DATA_PATH+'elo/', 'elo_*.csv'))))
        df=df[['Club', 'Country', 'Level', 'Elo', 'From', 'To']]
        df.columns=['team', 'country', 'level', 'elo', 'ds', 'de']
        df=self._encode_teams(df)
        return df

    def _provide_sofa(self):
        dp=SofaDataProvider(load=True)
        df=dp._load_data()
        return df.drop_duplicates(subset='mid', keep='last')

    def _provide_op(self):
        dp=OpDataProvider(load=True)
        df=dp._load_data()
        return df

    def _bind_sofa_op(self,df):
        df_op=self._provide_op()
        df_binds=pd.read_csv('data/binds_ss_op.csv', index_col=None)
        df_op=df_op.merge(df_binds[['op_mid','mid']], left_on='mid', right_on='op_mid')
        return df.merge(df_op[['mid_y','odds_away','odds_draw','odds_home','oddsprob_home','oddsprob_draw','oddsprob_away','drift_home','drift_away','drift_draw']], left_on='mid', right_on='mid_y', how='left')

    def _load_data(self):
        #df_op=self._provide_sofa()
        df_sofa=self._provide_sofa()
        df_elo=self._provide_elo()
        df_sofa=self._add_elo(df_sofa,df_elo)
        df_sofa=self._bind_sofa_op(df_sofa)
        return df_sofa
    
    def provide_data(self, double=True):
        df=self._load_data()
        df['psft']=df.sc1-df.sc2
        df['psht']=df.homeScoreHT-df.awayScoreHT
        df['w1']=np.where(df.winner=='home',1,0)
        df['wx']=np.where(df.winner=='draw',1,0)
        df['w2']=np.where(df.winner=='away',1,0)
        df_home=df.copy()
        df_home=df_home.rename(columns={'homeScoreHT':'ht1','awayScoreHT':'ht2','sc1':'ft1','sc2':'ft2','vote_home':'vote1','vote_draw':'votex','vote_away':'vote2','home_formation':'form1','away_formation':'form2','oddsprob_home':'oddsprob1','oddsprob_draw':'oddsprobx','oddsprob_away':'oddsprob2','drift_home':'drift1','drift_draw':'driftx','drift_away':'drift2'})
        if double:
            df_home['side']=1
            df_away=df.copy()
            df_away['side']=0
            df_away=df_away.rename(columns={'homeScoreHT':'ht2','awayScoreHT':'ht1','sc1':'ft2','sc2':'ft1','vote_home':'vote2','vote_draw':'votex','vote_away':'vote1',
                                            'home_formation':'form2','away_formation':'form1','w1':'w2','w2':'w1','elo1':'elo2','elo2':'elo1','t1':'t2','t2':'t1',
                                            'tid1':'tid2','tid2':'tid1','odds_away':'odds_home','odds_home':'odds_away','oddsprob1':'oddsprob2',
                                            'oddsprob2':'oddsprob1','drift1':'drift2','drift2':'drift1',
                                            'possession1':'possession2', 'shont1':'shont2', 'shofft1':'shofft2', 'corners1':'corners2', 
                                            'offsides1':'offsides2', 'fouls1':'fouls2', 'cards1':'cards2', 'gksaves1':'gksaves2',
                                            'possession2':'possession1', 'shont2':'shont1', 'shofft2':'shofft1', 'corners2':'corners1', 
                                            'offsides2':'offsides1', 'fouls2':'fouls1', 'cards2':'cards1', 'gksaves2':'gksaves1'})
            df_away['psft']=df_away['psft']*-1
            df_away['psht']=df_away['psht']*-1

            df_home=pd.concat([df_home,df_away], axis=0)

        return df_home.reset_index(drop=True)

In [17]:
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import pytz
from datetime import timezone,datetime,timedelta
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler
import gc

import api.util
from api.data_collector import DataCollector
from api.sofa_dp import SofaDataProvider
from api.op_dp import OpDataProvider

from IPython.display import display
pd.options.display.max_columns = None
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
dp=DataCollector()
#df=dp.provide_data(double=False)
df=dp.provide_data()

Empty DataFrame
Columns: [team]
Index: []


In [12]:
#['side', 'country', 'country_id', 'liga', 'mid', 'round', 'ds', 'de', 'tid1', 'tid2', 't1', 't2', 'w1', 'wx', 'w2',  'ht1', 'ht2', 'ft1', 'ft2', 'ps_ht', 'ps_ft', 
#       'winner', 'form1', 'form2', 'vote1', 'votex', 'vote2', 'pop_r', 'elo1', 'elo2']
COL_CUR=['country_id', 'round', 'ds', 'de', 'form1', 'form2', 'vote1', 'votex', 'vote2', 'pop_r', 'elo1', 'elo2','oddsprob1','oddsprobx','oddsprob2','drift1','drift2','driftx']
COL_PREV=['w1', 'wx', 'w2',  'ht1', 'ht2', 'ft1', 'ft2', 'psht', 'psft', 'graph1','graph2', 'possession1','possession2', 'shont1','shont2', 'shofft1','shofft2', 'corners1','corners2', 'offsides1','offsides2', 'fouls1','fouls2', 'cards1','cards2', 'gksaves1','gksaves2','precision1','precision2']
COL_CAT=['country_id','form1', 'form2', 'pop_r']
COL_BIN=['side']
COL_NUM=['w1', 'wx', 'w2',  'ht1', 'ht2', 'ft1', 'ft2', 'psht', 'psft', 'vote1', 'votex', 'vote2', 'elo1', 'elo2','oddsprob1','oddsprobx','oddsprob2','drift1','driftx','drift2', 'graph1','graph2', 'possession1','possession2', 'shont1','shont2', 'shofft1','shofft2', 'corners1','corners2', 'offsides1','offsides2', 'fouls1','fouls2', 'cards1','cards2', 'gksaves1','gksaves2','precision1','precision2']
COL_INF=['side','country', 'liga', 'mid', 'round', 'ds', 't1', 't2','tid1', 'tid2', 'w1', 'wx', 'w2',  'ft1', 'ft2','winner','odds_away','odds_draw','odds_home']

#df=pd.read_csv('data/matches.csv', index_col=None)
#df['elo1'].fillna((df['elo1'].mean()), inplace=True)
#df['elo2'].fillna((df['elo2'].mean()), inplace=True)


In [13]:
def calc_stat(df_, group_feature, name, cols):
    df_=df_.sort_values(by='ds')
    cols_out=['ds','mid','tid']
    df_['C']=df_.groupby([group_feature])['N'].apply(lambda x : x.shift().cumsum())#.cumsum()#-1
    #display(df_)
    for col in cols:
        print(col, end=', ')
        cols_out.append(f'{col}_{name}_avg')
        df_[f'{col}_{name}_sum']=df_.groupby([group_feature])[col].apply(lambda x : x.shift().cumsum())#.cumsum()#-df_teams[col]
        df_[f'{col}_{name}_avg']=df_[f'{col}_{name}_sum']/df_['C']
        df_.drop(columns=[f'{col}_{name}_sum'], inplace=True)
        #cols_out.append(f'{col}_{name}_sum')
    df_=df_.rename(columns={group_feature:'tid'})
    return df_[cols_out]

def calc_stat_n(df_, group_feature, name, cols, n):
    df_=df_.sort_values(by='ds')
    cols_out=['ds','mid','tid']
    df_['C']=df_.groupby([group_feature])['N'].apply(lambda x : x.shift().rolling(min_periods=1, window=n).sum())
    #display(df_)
    for col in cols:
        print(col, end=', ')
        cols_out.append(f'{col}_{name}_{n}')
        df_[f'{col}_{name}_sum']=df_.groupby([group_feature])[col].apply(lambda x : x.shift().rolling(min_periods=1, window=n).sum())
        df_[f'{col}_{name}_{n}']=df_[f'{col}_{name}_sum']/df_['C']
        df_.drop(columns=[f'{col}_{name}_sum'], inplace=True)
        #cols_out.append(f'{col}_{name}_sum')
    df_=df_.rename(columns={group_feature:'tid'})
    return df_[cols_out]

In [14]:
deep=3
df['N']=1
cols_cum=COL_NUM
df_all=calc_stat(df, 'tid1', 'tt',cols_cum).sort_values(by=['mid','tid'])
df_all3=calc_stat_n(df, 'tid1', 'tt',cols_cum, deep).sort_values(by=['mid','tid'])
colnames=[x.replace('_avg','_form') for x in df_all.columns[3:]]
res=df_all3[df_all3.columns[3:]].values-df_all[df_all.columns[3:]].values
df_form=pd.DataFrame(res,columns=colnames)
df_all=pd.concat([df_all,df_form], axis=1)

df_home=calc_stat(df[df['side']==1], 'tid1', 'ts', cols_cum).sort_values(by=['mid','tid'])
df_home3=calc_stat_n(df[df['side']==1], 'tid1', 'ts', cols_cum, deep).sort_values(by=['mid','tid'])
colnames=[x.replace('_avg','_form') for x in df_home.columns[3:]]
res=df_home3[df_home3.columns[3:]].values-df_home[df_home.columns[3:]].values
df_form=pd.DataFrame(res,columns=colnames)
df_home=pd.concat([df_home,df_form], axis=1)

df_away=calc_stat(df[df['side']==0], 'tid1', 'ts', cols_cum).reset_index(drop=True).sort_values(by=['mid','tid'])
df_away3=calc_stat_n(df[df['side']==0], 'tid1', 'ts', cols_cum, deep).sort_values(by=['mid','tid'])
colnames=[x.replace('_avg','_form') for x in df_away.columns[3:]]
res=df_away3[df_away3.columns[3:]].values-df_away[df_away.columns[3:]].values
df_form=pd.DataFrame(res,columns=colnames)
df_away=pd.concat([df_away,df_form], axis=1)

#df_home=calc_stat(df, 'tid1', 'th', cols_cum)
#df_away=calc_stat(df, 'tid2', 'ta', cols_cum)

df_all.to_csv('data/teams_stats_all.csv', index=False)
df_home.to_csv('data/teams_stats_home.csv', index=False)
df_away.to_csv('data/teams_stats_away.csv', index=False)
df_all3.to_csv('data/teams_stats_all3.csv', index=False)
df_home3.to_csv('data/teams_stats_home3.csv', index=False)
df_away3.to_csv('data/teams_stats_away3.csv', index=False)

w1
wx
w2
ht1
ht2
ft1
ft2
psht
psft
vote1
votex
vote2
elo1
elo2
oddsprob1
oddsprobx
oddsprob2
drift1
driftx
drift2
graph1
graph2
possession1
possession2
shont1
shont2
shofft1
shofft2
corners1
corners2
offsides1
offsides2
fouls1
fouls2
cards1
cards2
gksaves1
gksaves2
precision1
precision2
w1
wx
w2
ht1
ht2
ft1
ft2
psht
psft
vote1
votex
vote2
elo1
elo2
oddsprob1
oddsprobx
oddsprob2
drift1
driftx
drift2
graph1
graph2
possession1
possession2
shont1
shont2
shofft1
shofft2
corners1
corners2
offsides1
offsides2
fouls1
fouls2
cards1
cards2
gksaves1
gksaves2
precision1
precision2
w1
wx
w2
ht1
ht2
ft1
ft2
psht
psft
vote1
votex
vote2
elo1
elo2
oddsprob1
oddsprobx
oddsprob2
drift1
driftx
drift2
graph1
graph2
possession1
possession2
shont1
shont2
shofft1
shofft2
corners1
corners2
offsides1
offsides2
fouls1
fouls2
cards1
cards2
gksaves1
gksaves2
precision1
precision2
w1
wx
w2
ht1
ht2
ft1
ft2
psht
psft
vote1
votex
vote2
elo1
elo2
oddsprob1
oddsprobx
oddsprob2
drift1
driftx
drift2
graph1
graph2
possessi

df_all=pd.read_csv('data/teams_stats_all.csv', index_col=None)
df_home=pd.read_csv('data/teams_stats_home.csv', index_col=None)
df_away=pd.read_csv('data/teams_stats_away.csv', index_col=None)

df_all=pd.read_csv('data/teams_stats_all.csv', index_col=None)
df_home=pd.read_csv('data/teams_stats_home.csv', index_col=None)
df_away=pd.read_csv('data/teams_stats_away.csv', index_col=None)

df_all.columns

id=9270007
tid1=594
tid2=1499
display(df_all3[df_all3.mid==id])
display(df_home3[df_home3.mid==id])
display(df_away3[df_away3.mid==id])


### df_all
- `tt_avg` - Все матчи до текущего, среднее
- `tt_form` - Среднее по 3 последним, без `tt_avg`

### df_home
Статсы только тех игр, когда команда играла дома
- `ts_avg` - Все матчи до текущего, среднее
- `ts_form` - Среднее по 3 последним, без `ts_avg`

### df_away
Статсы только тех игр, когда команда играла не дома
- `ts_avg` - Все матчи до текущего, среднее
- `ts_form` - Среднее по 3 последним, без `ts_avg`

Все, что 1 относится к самой команде, а с 2 - к её оппонентам, объединив данные добавлено два префикса - `tar_` и `_opp` соответственно.

Нужно скопировать статсы по командам с учетом:
1. Таргетная команда и оппонент.
2. Дома или нет.

## Вычисление diffs
Что есть 1 и 2 для разных статсов? 

На примере w1 - wx - w2:
- tar_w1_tt_avg - среднее побед таргетной команды по всем играм
- tar_wx_tt_avg - среднее ничей таргетной команды по всем играм
- tar_w2_tt_avg - среднее проигрышей таргетной команды по всем играм
- opp_w1_tt_avg - среднее побед оппонента по всем играм
- opp_wx_tt_avg - среднее ничей оппонента по всем играм
- opp_w2_tt_avg - среднее проигрышей оппонента по всем играм

tar_w1_tt_form - разница между последними играми и всей статистикой, может быть как положительной, если в последние игры параметр больше, так и отрицательной

аналогично с ts, только учитывается статистика только по играм дома либо в гостях

## Перекрестные сравнения: забил - пропустил
- tar_ft1_tt_avg - среднее количество голов забила таргетная команда по всем играм (оказала давление - graph1)
- tar_ft2_tt_avg - среднее количество голов пропустила таргетная команда по всем играм (позволила давление - graph2)
- opp_ft1_tt_avg - среднее количество голов забила команда оппонента по всем играм
- opp_ft2_tt_avg - среднее количество голов пропустила команда оппонента по всем играм

tar_ft1_tt_avg и opp_ft2_tt_avg => `ft1_tt_avg`, tar_ft2_tt_avg и opp_ft1_tt_avg => `ft2_tt_avg`, а также ft1_tt_avg - ft2_tt_avg => `diff_ft_tt_avg`

tar_graph1_tt_avg и opp_graph2_tt_avg => `graph1_tt_avg`, tar_graph2_tt_avg и opp_graph1_tt_avg => `graph2_tt_avg`, а также graph1_tt_avg - graph2_tt_avg => `diff_graph_tt_avg`

tar_w1_tt_avg и opp_w2_tt_avg => `w1_tt_avg`, tar_w2_tt_avg и opp_w1_tt_avg => `w2_tt_avg`, tar_wx_tt_avg и opp_wx_tt_avg => `wx_tt_avg`, а также w1_tt_avg - w2_tt_avg => `diff_w_tt_avg`

tar_psft_tt_avg и opp_psft_tt_avg => `psft_tt_avg`

In [15]:
# Копируем данные всех игр, и домашних и нет, вперемешку. Это - таргетный датасет.
# Префикс для него - tar_, а суффикс - _tt_.
df_all_target=df_all.copy()
df_all_target.columns=np.hstack([df_all.columns[:3],['tar_'+x for x in df_all.columns[3:]]])

# Точно такой же датасет называем оппонентным.
# Префикс для него - opp_, а суффикс - _tt_.
df_all_opponent=df_all.copy()
df_all_opponent.columns=np.hstack([df_all.columns[:3],['opp_'+x for x in df_all.columns[3:]]])

# Для команд, игравших дома делаем датасет из домашнего (домашний таргетный). 
# Префикс для него - tar_, а суффикс - _ts_.
df_home_target=df_home.copy()
df_home_target.columns=np.hstack([df_home.columns[:3],['tar_'+x.replace('_th_','_ts_') for x in df_home.columns[3:]]])
# Для их противников делаем датасет из гостевого (гостевой оппонентный). 
# Префикс для него - opp_, а суффикс - _ts_.
df_away_opponent=df_away.copy()
df_away_opponent.columns=np.hstack([df_away.columns[:3],['opp_'+x.replace('_ta_','_ts_') for x in df_away.columns[3:]]])

# Для команд, игравших в гостях делаем датасет из гостевого (гостевой таргетный). 
# Префикс для него - tar_, а суффикс - _ts_.
df_away_target=df_away.copy()
df_away_target.columns=np.hstack([df_away.columns[:3],['tar_'+x.replace('_ta_','_ts_') for x in df_away.columns[3:]]])
# Для их противников делаем датасет из домашнего (домашний оппонентный). 
# Префикс для него - opp_, а суффикс - _ts_.
df_home_opponent=df_home.copy()
df_home_opponent.columns=np.hstack([df_home.columns[:3],['opp_'+x.replace('_th_','_ts_') for x in df_home.columns[3:]]])

# Итого есть 6 датасетов с шаблонными названиями переменных:
# tar_???_tt_, opp_???_tt_ - для статсов по всем играм
# tar_???_ts_, opp_???_ts_ - для статсов с учётом дома / в гостях для таргетных команд, игравших дома
# tar_???_ts_, opp_???_ts_ - для статсов с учётом дома / в гостях для таргетных команд, игравших в гостях 

# Делаем два датасета для таргетных домашних и гостевых команд.
# В данные берём лишь информационные поля (по которым потом проводить анализ) - COL_INF, 
# и текущие данные, доступные к сбору перед матчем - COL_CUR. 
df_1=df[df['side']==1][COL_INF+COL_CUR]
df_2=df[df['side']==0][COL_INF+COL_CUR]

# Заполняем данные для таргетных команд, играющих дома:
# Статсы по всем данным для первой команды (таргет)                     tar_???_tt_
df_1=df_1.merge(df_all_target[df_all_target.columns[1:]], left_on=['mid','tid1'], right_on=['mid','tid'], how='left')
# Статсы по всем данным для второй команды (оппонент)                   opp_???_tt_
df_1=df_1.merge(df_all_opponent[df_all_opponent.columns[1:]], left_on=['mid','tid2'], right_on=['mid','tid'], how='left')
# Статсы по всем играм дома для первой команды (таргет домашняя)        tar_???_ts_
df_1=df_1.merge(df_home_target[df_home_target.columns[1:]], left_on=['mid','tid1'], right_on=['mid','tid'], how='left')
# Статсы по всем играм в гостях для второй команды (оппонент гостевая)  opp_???_ts_
df_1=df_1.merge(df_away_opponent[df_away_opponent.columns[1:]], left_on=['mid','tid2'], right_on=['mid','tid'], how='left')

# То же для таргетных команд, играющих в гостях:
# Статсы по всем данным для первой команды (таргет)                     tar_???_tt_
df_2=df_2.merge(df_all_target[df_all_target.columns[1:]], left_on=['mid','tid1'], right_on=['mid','tid'], how='left')
# Статсы по всем данным для второй команды (оппонент)                   opp_???_tt_
df_2=df_2.merge(df_all_opponent[df_all_opponent.columns[1:]], left_on=['mid','tid2'], right_on=['mid','tid'], how='left')
# Статсы по всем играм в гостях для первой команды (таргет гостевая)    tar_???_ts_
df_2=df_2.merge(df_away_target[df_away_target.columns[1:]], left_on=['mid','tid1'], right_on=['mid','tid'], how='left')
# Статсы по всем играм дома для второй команды (оппонент домашняя)      opp_???_ts_
df_2=df_2.merge(df_home_opponent[df_home_opponent.columns[1:]], left_on=['mid','tid2'], right_on=['mid','tid'], how='left')

# Объединяем оба датасета и удаляем дубликаты колонок:
df_=pd.concat([df_1,df_2], axis=0)
df_.drop(columns=['tid_x','tid_y'], inplace=True)

# Очищаем память
del df_all_target,df_all_opponent,df_home_target,df_away_opponent,df_away_target,df_home_opponent,df_1,df_2
gc.collect()

0

# Diffs
COL_PREV=['w1', 'wx', 'w2',  'ht1', 'ht2', 'ft1', 'ft2', 'ps_ht', 'ps_ft', 'graph1','graph2', 'possession1','possession2', 'shont1','shont2', 'shofft1','shofft2', 'corners1','corners2', 'offsides1','offsides2', 'fouls1','fouls2', 'cards1','cards2', 'gksaves1','gksaves2','precision1','precision2']

cols_tar=[x for x in df_.columns if 'tar_' in x]
cols_opp=[x for x in df_.columns if 'opp_' in x]
cols_diff=[x.replace('tar_','_diff_') for x in cols_tar]
df_.reset_index(drop=True, inplace=True)
df_=pd.concat([df_,pd.DataFrame(df_[cols_tar].values-df_[cols_opp].values, columns=cols_diff)], axis=1)
df_['diff_vote12']=df_['vote1']-df_['vote2']
df_['diff_elo']=df_['elo1']-df_['elo2']
df_['diff_op']=df_['oddsprob_home']-df_['oddsprob_away']
df_.to_csv('data/stats_generated1.csv', index=False)

In [16]:
# Diffs
def diff_numbers(df, feature):
    print(feature,', ', end='')
    for scope in ['tt','ts']:
        for typ in ['avg','form']:
            col1=f'tar_{feature}1_{scope}_{typ}'
            col2=f'opp_{feature}2_{scope}_{typ}'
            col_out1=f'{feature}1_{scope}_{typ}'
            df[col_out1]=(df[col1]+df[col2])/2
            col1=f'tar_{feature}2_{scope}_{typ}'
            col2=f'opp_{feature}1_{scope}_{typ}'
            col_out2=f'{feature}2_{scope}_{typ}'
            df[col_out2]=(df[col1]+df[col2])/2
            col_out=f'diff_{feature}_{scope}_{typ}'
            df[col_out]=df[col_out1]-df[col_out2]
    return df

def diff_non_numbers(df, feature):
    print(feature,', ', end='')
    for scope in ['tt','ts']:
        for typ in ['avg','form']:
            'tar_wx_tt_avg'
            col1=f'tar_{feature}_{scope}_{typ}'
            col2=f'opp_{feature}_{scope}_{typ}'
            col_out=f'{feature}_{scope}_{typ}'
            df[col_out]=(df[col1]+df[col2])/2
    return df

features=['w',  'ht', 'ft', 'graph', 'possession', 'shont', 'shofft', 'corners', 'offsides', 'fouls','cards', 'gksaves','precision']
features_non_numbers=['wx',  'psht', 'psft']

df=df_.copy()
for col in features:
    df=diff_numbers(df, col)

print('')
for col in features_non_numbers:
    df=diff_non_numbers(df, col)


w , ht , ft , graph , possession , shont , shofft , corners , offsides , fouls , cards , gksaves , precision , 
wx , psht , psft , 

In [17]:
df['diff_vote12']=df['vote1']-df_['vote2']
df['diff_elo']=df['elo1']-df_['elo2']
df['diff_op']=df['oddsprob1']-df_['oddsprob2']
df.to_csv('data/stats_generated.csv', index=False)

# Teams last stats

In [71]:
df_all=pd.read_csv('data/stats_generated.csv', index_col=None)
df_all=df_all.dropna(subset=['ft1','ft2'])
df_all['ds']=df_all['ds']=pd.to_datetime(df_all['ds'])
df_all=df_all.sort_values(by='ds')
df_last=df_all.drop_duplicates(subset=['side','tid1'], keep='last')
df_last.to_csv('data/stats_teams.csv', index=False)

# Today

In [76]:
from api.data_collector import DataCollector
dp=DataCollector(today=True)
#df=dp.provide_data()
df=dp.provide_today()
df_last=pd.read_csv('data/stats_teams.csv', index_col=None)

93
Empty DataFrame
Columns: [team]
Index: []


In [77]:
df

,ht2,country,country_id,ds,ht1,liga,mid,round,ft1,ft2,t1,t2,tid1,tid2,winner,formation_h,formation_a,form1,form2,vote1,votex,vote2,votes,y,pop_r,de,elo1,elo2,op_tid1,op_tid2,odds_away,odds_draw,odds_home,oddsprob1,oddsprobx,oddsprob2,drift1,drift2,driftx,psft,psht,w1,wx,w2,side,awayScoreHT,homeScoreHT,sc1,sc2,oddsprob_home,oddsprob_draw,oddsprob_away,drift_home,drift_away,drift_draw
0,NaN,england,15,2021-01-31 12:00:00+00:00,NaN,premier-league,8897050,21,NaN,NaN,chelsea,burnley,4,3,draw,NaN,NaN,NaN,NaN,0.762910,0.160065,0.077026,27770.0,2021,4.0,2021-01-31,1828.057861,1719.795166,256,135,9.68,5.20,1.35,0.713223,0.186128,0.100649,-0.068265,0.210811,0.081265,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,england,15,2021-01-31 14:00:00+00:00,NaN,premier-league,8897051,21,NaN,NaN,leicester city,leeds united,6,796,draw,4-2-3-1,4-1-4-1,14.0,12.0,0.689995,0.206004,0.104001,23019.0,2021,3.0,2021-01-31,1821.147705,1676.518799,239,1503,4.25,4.05,1.81,0.533874,0.238475,0.227651,-0.041660,0.058843,0.025857,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,england,15,2021-01-31 16:30:00+00:00,NaN,premier-league,8897027,21,NaN,NaN,west ham united,liverpool,120,181,draw,4-2-3-1,4-3-3,14.0,18.0,0.193081,0.186881,0.620038,28066.0,2021,4.0,2021-01-31,1757.818237,1944.577148,444,1183,1.78,4.04,4.41,0.218530,0.240785,0.540685,-0.091515,0.037856,-0.022932,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,england,15,2021-01-31 19:15:00+00:00,NaN,premier-league,8897039,21,NaN,NaN,brighton hove albion,tottenham,609,274,draw,3-5-2,4-2-3-1,9.0,14.0,0.119534,0.192975,0.687491,21609.0,2021,3.0,2021-01-31,1662.046997,1836.129150,984,680,2.28,3.40,3.30,0.291167,0.286855,0.421978,-0.078422,0.078682,-0.059461,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,spain,42,2021-01-31 15:15:00+00:00,NaN,laliga,8966464,21,NaN,NaN,cadiz,atletico madrid,940,134,draw,4-4-2,4-4-2,20.0,20.0,0.126301,0.173107,0.700591,22668.0,2021,3.0,2021-01-31,1598.655518,1927.645752,1588,261,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,NaN,austria,2,2021-01-31 13:30:00+00:00,NaN,bundesliga,8858506,15,NaN,NaN,wolfsberger ac,skn st polten,330,1033,draw,NaN,NaN,NaN,NaN,0.457300,0.347658,0.195041,1815.0,2021,0.0,2021-01-31,1550.491699,1419.237427,1395,368,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
154,NaN,turkey,45,2021-01-31 13:00:00+00:00,NaN,tff-1-lig,8917288,19,NaN,NaN,adana demirspor,boluspor,153,410,draw,NaN,NaN,NaN,NaN,0.509294,0.267658,0.223048,807.0,2021,0.0,2021-01-31,NaN,NaN,6537,4729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
155,NaN,belgium,4,2021-01-31 12:30:00+00:00,NaN,pro-league,8833161,23,NaN,NaN,standard liege,club brugge,270,82,draw,NaN,NaN,NaN,NaN,0.081898,0.254644,0.663458,7375.0,2021,1.0,2021-01-31,1521.353760,1688.555908,562,1355,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
156,NaN,germany,20,2021-01-31 13:00:00+00:00,NaN,3-liga,8897913,22,NaN,NaN,fsv zwickau,tsv 1860 munchen,1161,296,draw,NaN,NaN,NaN,NaN,0.052140,0.254796,0.693064,2033.0,2021,0.0,2021-01-31,NaN,NaN,1193,2159,1.84,3.37,4.47,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0,NaN,NaN,NaN,NaN,0.505812,0.280372,0.213816,0.063097,-0.084783,-0.082277


In [79]:
df_last[(df_last['side']==1) & (df_last['tid1']==4)]

,side,country,liga,mid,round,ds,t1,t2,tid1,tid2,w1,wx,w2,ft1,ft2,winner,odds_away,odds_draw,odds_home,country_id,round.1,ds.1,de,form1,form2,vote1,votex,vote2,pop_r,elo1,elo2,oddsprob1,oddsprobx,oddsprob2,drift1,drift2,driftx,tar_w1_tt_avg,tar_wx_tt_avg,tar_w2_tt_avg,tar_ht1_tt_avg,tar_ht2_tt_avg,tar_ft1_tt_avg,tar_ft2_tt_avg,tar_psht_tt_avg,tar_psft_tt_avg,tar_vote1_tt_avg,tar_votex_tt_avg,tar_vote2_tt_avg,tar_elo1_tt_avg,tar_elo2_tt_avg,tar_oddsprob1_tt_avg,tar_oddsprobx_tt_avg,tar_oddsprob2_tt_avg,tar_drift1_tt_avg,tar_driftx_tt_avg,tar_drift2_tt_avg,tar_graph1_tt_avg,tar_graph2_tt_avg,tar_possession1_tt_avg,tar_possession2_tt_avg,tar_shont1_tt_avg,tar_shont2_tt_avg,tar_shofft1_tt_avg,tar_shofft2_tt_avg,tar_corners1_tt_avg,tar_corners2_tt_avg,tar_offsides1_tt_avg,tar_offsides2_tt_avg,tar_fouls1_tt_avg,tar_fouls2_tt_avg,tar_cards1_tt_avg,tar_cards2_tt_avg,tar_gksaves1_tt_avg,tar_gksaves2_tt_avg,tar_precision1_tt_avg,tar_precision2_tt_avg,tar_w1_tt_form,tar_wx_tt_form,tar_w2_tt_form,tar_ht1_tt_form,tar_ht2_tt_form,tar_ft1_tt_form,tar_ft2_tt_form,tar_psht_tt_form,tar_psft_tt_form,tar_vote1_tt_form,tar_votex_tt_form,tar_vote2_tt_form,tar_elo1_tt_form,tar_elo2_tt_form,tar_oddsprob1_tt_form,tar_oddsprobx_tt_form,tar_oddsprob2_tt_form,tar_drift1_tt_form,tar_driftx_tt_form,tar_drift2_tt_form,tar_graph1_tt_form,tar_graph2_tt_form,tar_possession1_tt_form,tar_possession2_tt_form,tar_shont1_tt_form,tar_shont2_tt_form,tar_shofft1_tt_form,tar_shofft2_tt_form,tar_corners1_tt_form,tar_corners2_tt_form,tar_offsides1_tt_form,tar_offsides2_tt_form,tar_fouls1_tt_form,tar_fouls2_tt_form,tar_cards1_tt_form,tar_cards2_tt_form,tar_gksaves1_tt_form,tar_gksaves2_tt_form,tar_precision1_tt_form,tar_precision2_tt_form,opp_w1_tt_avg,opp_wx_tt_avg,opp_w2_tt_avg,opp_ht1_tt_avg,opp_ht2_tt_avg,opp_ft1_tt_avg,opp_ft2_tt_avg,opp_psht_tt_avg,opp_psft_tt_avg,opp_vote1_tt_avg,opp_votex_tt_avg,opp_vote2_tt_avg,opp_elo1_tt_avg,opp_elo2_tt_avg,opp_oddsprob1_tt_avg,opp_oddsprobx_tt_avg,opp_oddsprob2_tt_avg,opp_drift1_tt_avg,opp_driftx_tt_avg,opp_drift2_tt_avg,opp_graph1_tt_avg,opp_graph2_tt_avg,opp_possession1_tt_avg,opp_possession2_tt_avg,opp_shont1_tt_avg,opp_shont2_tt_avg,opp_shofft1_tt_avg,opp_shofft2_tt_avg,opp_corners1_tt_avg,opp_corners2_tt_avg,opp_offsides1_tt_avg,opp_offsides2_tt_avg,opp_fouls1_tt_avg,opp_fouls2_tt_avg,opp_cards1_tt_avg,opp_cards2_tt_avg,opp_gksaves1_tt_avg,opp_gksaves2_tt_avg,opp_precision1_tt_avg,opp_precision2_tt_avg,opp_w1_tt_form,opp_wx_tt_form,opp_w2_tt_form,opp_ht1_tt_form,opp_ht2_tt_form,opp_ft1_tt_form,opp_ft2_tt_form,opp_psht_tt_form,opp_psft_tt_form,opp_vote1_tt_form,opp_votex_tt_form,opp_vote2_tt_form,opp_elo1_tt_form,opp_elo2_tt_form,opp_oddsprob1_tt_form,opp_oddsprobx_tt_form,opp_oddsprob2_tt_form,opp_drift1_tt_form,opp_driftx_tt_form,opp_drift2_tt_form,opp_graph1_tt_form,opp_graph2_tt_form,opp_possession1_tt_form,opp_possession2_tt_form,opp_shont1_tt_form,opp_shont2_tt_form,opp_shofft1_tt_form,opp_shofft2_tt_form,opp_corners1_tt_form,opp_corners2_tt_form,opp_offsides1_tt_form,opp_offsides2_tt_form,opp_fouls1_tt_form,opp_fouls2_tt_form,opp_cards1_tt_form,opp_cards2_tt_form,opp_gksaves1_tt_form,opp_gksaves2_tt_form,opp_precision1_tt_form,opp_precision2_tt_form,tar_w1_ts_avg,tar_wx_ts_avg,tar_w2_ts_avg,tar_ht1_ts_avg,tar_ht2_ts_avg,tar_ft1_ts_avg,tar_ft2_ts_avg,tar_psht_ts_avg,tar_psft_ts_avg,tar_vote1_ts_avg,tar_votex_ts_avg,tar_vote2_ts_avg,tar_elo1_ts_avg,tar_elo2_ts_avg,tar_oddsprob1_ts_avg,tar_oddsprobx_ts_avg,tar_oddsprob2_ts_avg,tar_drift1_ts_avg,tar_driftx_ts_avg,tar_drift2_ts_avg,tar_graph1_ts_avg,tar_graph2_ts_avg,tar_possession1_ts_avg,tar_possession2_ts_avg,tar_shont1_ts_avg,tar_shont2_ts_avg,tar_shofft1_ts_avg,tar_shofft2_ts_avg,tar_corners1_ts_avg,tar_corners2_ts_avg,tar_offsides1_ts_avg,tar_offsides2_ts_avg,tar_fouls1_ts_avg,tar_fouls2_ts_avg,tar_cards1_ts_avg,tar_cards2_ts_avg,tar_gksaves1_ts_avg,tar_gksaves2_ts_avg,tar_precision1_ts_avg,tar_precision2_ts_avg,tar_w1_ts_form,tar_wx_ts_form,tar_w2_ts_form,tar_ht1_t

In [80]:
df_last[(df_last['side']==0) & (df_last['tid1']==3)]

,side,country,liga,mid,round,ds,t1,t2,tid1,tid2,w1,wx,w2,ft1,ft2,winner,odds_away,odds_draw,odds_home,country_id,round.1,ds.1,de,form1,form2,vote1,votex,vote2,pop_r,elo1,elo2,oddsprob1,oddsprobx,oddsprob2,drift1,drift2,driftx,tar_w1_tt_avg,tar_wx_tt_avg,tar_w2_tt_avg,tar_ht1_tt_avg,tar_ht2_tt_avg,tar_ft1_tt_avg,tar_ft2_tt_avg,tar_psht_tt_avg,tar_psft_tt_avg,tar_vote1_tt_avg,tar_votex_tt_avg,tar_vote2_tt_avg,tar_elo1_tt_avg,tar_elo2_tt_avg,tar_oddsprob1_tt_avg,tar_oddsprobx_tt_avg,tar_oddsprob2_tt_avg,tar_drift1_tt_avg,tar_driftx_tt_avg,tar_drift2_tt_avg,tar_graph1_tt_avg,tar_graph2_tt_avg,tar_possession1_tt_avg,tar_possession2_tt_avg,tar_shont1_tt_avg,tar_shont2_tt_avg,tar_shofft1_tt_avg,tar_shofft2_tt_avg,tar_corners1_tt_avg,tar_corners2_tt_avg,tar_offsides1_tt_avg,tar_offsides2_tt_avg,tar_fouls1_tt_avg,tar_fouls2_tt_avg,tar_cards1_tt_avg,tar_cards2_tt_avg,tar_gksaves1_tt_avg,tar_gksaves2_tt_avg,tar_precision1_tt_avg,tar_precision2_tt_avg,tar_w1_tt_form,tar_wx_tt_form,tar_w2_tt_form,tar_ht1_tt_form,tar_ht2_tt_form,tar_ft1_tt_form,tar_ft2_tt_form,tar_psht_tt_form,tar_psft_tt_form,tar_vote1_tt_form,tar_votex_tt_form,tar_vote2_tt_form,tar_elo1_tt_form,tar_elo2_tt_form,tar_oddsprob1_tt_form,tar_oddsprobx_tt_form,tar_oddsprob2_tt_form,tar_drift1_tt_form,tar_driftx_tt_form,tar_drift2_tt_form,tar_graph1_tt_form,tar_graph2_tt_form,tar_possession1_tt_form,tar_possession2_tt_form,tar_shont1_tt_form,tar_shont2_tt_form,tar_shofft1_tt_form,tar_shofft2_tt_form,tar_corners1_tt_form,tar_corners2_tt_form,tar_offsides1_tt_form,tar_offsides2_tt_form,tar_fouls1_tt_form,tar_fouls2_tt_form,tar_cards1_tt_form,tar_cards2_tt_form,tar_gksaves1_tt_form,tar_gksaves2_tt_form,tar_precision1_tt_form,tar_precision2_tt_form,opp_w1_tt_avg,opp_wx_tt_avg,opp_w2_tt_avg,opp_ht1_tt_avg,opp_ht2_tt_avg,opp_ft1_tt_avg,opp_ft2_tt_avg,opp_psht_tt_avg,opp_psft_tt_avg,opp_vote1_tt_avg,opp_votex_tt_avg,opp_vote2_tt_avg,opp_elo1_tt_avg,opp_elo2_tt_avg,opp_oddsprob1_tt_avg,opp_oddsprobx_tt_avg,opp_oddsprob2_tt_avg,opp_drift1_tt_avg,opp_driftx_tt_avg,opp_drift2_tt_avg,opp_graph1_tt_avg,opp_graph2_tt_avg,opp_possession1_tt_avg,opp_possession2_tt_avg,opp_shont1_tt_avg,opp_shont2_tt_avg,opp_shofft1_tt_avg,opp_shofft2_tt_avg,opp_corners1_tt_avg,opp_corners2_tt_avg,opp_offsides1_tt_avg,opp_offsides2_tt_avg,opp_fouls1_tt_avg,opp_fouls2_tt_avg,opp_cards1_tt_avg,opp_cards2_tt_avg,opp_gksaves1_tt_avg,opp_gksaves2_tt_avg,opp_precision1_tt_avg,opp_precision2_tt_avg,opp_w1_tt_form,opp_wx_tt_form,opp_w2_tt_form,opp_ht1_tt_form,opp_ht2_tt_form,opp_ft1_tt_form,opp_ft2_tt_form,opp_psht_tt_form,opp_psft_tt_form,opp_vote1_tt_form,opp_votex_tt_form,opp_vote2_tt_form,opp_elo1_tt_form,opp_elo2_tt_form,opp_oddsprob1_tt_form,opp_oddsprobx_tt_form,opp_oddsprob2_tt_form,opp_drift1_tt_form,opp_driftx_tt_form,opp_drift2_tt_form,opp_graph1_tt_form,opp_graph2_tt_form,opp_possession1_tt_form,opp_possession2_tt_form,opp_shont1_tt_form,opp_shont2_tt_form,opp_shofft1_tt_form,opp_shofft2_tt_form,opp_corners1_tt_form,opp_corners2_tt_form,opp_offsides1_tt_form,opp_offsides2_tt_form,opp_fouls1_tt_form,opp_fouls2_tt_form,opp_cards1_tt_form,opp_cards2_tt_form,opp_gksaves1_tt_form,opp_gksaves2_tt_form,opp_precision1_tt_form,opp_precision2_tt_form,tar_w1_ts_avg,tar_wx_ts_avg,tar_w2_ts_avg,tar_ht1_ts_avg,tar_ht2_ts_avg,tar_ft1_ts_avg,tar_ft2_ts_avg,tar_psht_ts_avg,tar_psft_ts_avg,tar_vote1_ts_avg,tar_votex_ts_avg,tar_vote2_ts_avg,tar_elo1_ts_avg,tar_elo2_ts_avg,tar_oddsprob1_ts_avg,tar_oddsprobx_ts_avg,tar_oddsprob2_ts_avg,tar_drift1_ts_avg,tar_driftx_ts_avg,tar_drift2_ts_avg,tar_graph1_ts_avg,tar_graph2_ts_avg,tar_possession1_ts_avg,tar_possession2_ts_avg,tar_shont1_ts_avg,tar_shont2_ts_avg,tar_shofft1_ts_avg,tar_shofft2_ts_avg,tar_corners1_ts_avg,tar_corners2_ts_avg,tar_offsides1_ts_avg,tar_offsides2_ts_avg,tar_fouls1_ts_avg,tar_fouls2_ts_avg,tar_cards1_ts_avg,tar_cards2_ts_avg,tar_gksaves1_ts_avg,tar_gksaves2_ts_avg,tar_precision1_ts_avg,tar_precision2_ts_avg,tar_w1_ts_form,tar_wx_ts_form,tar_w2_ts_form,tar_ht1_t

In [73]:
df.merge(df_last)


,side,country,liga,mid,round,ds,t1,t2,tid1,tid2,w1,wx,w2,ft1,ft2,winner,odds_away,odds_draw,odds_home,country_id,round.1,ds.1,de,form1,form2,vote1,votex,vote2,pop_r,elo1,elo2,oddsprob1,oddsprobx,oddsprob2,drift1,drift2,driftx,tar_w1_tt_avg,tar_wx_tt_avg,tar_w2_tt_avg,tar_ht1_tt_avg,tar_ht2_tt_avg,tar_ft1_tt_avg,tar_ft2_tt_avg,tar_psht_tt_avg,tar_psft_tt_avg,tar_vote1_tt_avg,tar_votex_tt_avg,tar_vote2_tt_avg,tar_elo1_tt_avg,tar_elo2_tt_avg,tar_oddsprob1_tt_avg,tar_oddsprobx_tt_avg,tar_oddsprob2_tt_avg,tar_drift1_tt_avg,tar_driftx_tt_avg,tar_drift2_tt_avg,tar_graph1_tt_avg,tar_graph2_tt_avg,tar_possession1_tt_avg,tar_possession2_tt_avg,tar_shont1_tt_avg,tar_shont2_tt_avg,tar_shofft1_tt_avg,tar_shofft2_tt_avg,tar_corners1_tt_avg,tar_corners2_tt_avg,tar_offsides1_tt_avg,tar_offsides2_tt_avg,tar_fouls1_tt_avg,tar_fouls2_tt_avg,tar_cards1_tt_avg,tar_cards2_tt_avg,tar_gksaves1_tt_avg,tar_gksaves2_tt_avg,tar_precision1_tt_avg,tar_precision2_tt_avg,tar_w1_tt_form,tar_wx_tt_form,tar_w2_tt_form,tar_ht1_tt_form,tar_ht2_tt_form,tar_ft1_tt_form,tar_ft2_tt_form,tar_psht_tt_form,tar_psft_tt_form,tar_vote1_tt_form,tar_votex_tt_form,tar_vote2_tt_form,tar_elo1_tt_form,tar_elo2_tt_form,tar_oddsprob1_tt_form,tar_oddsprobx_tt_form,tar_oddsprob2_tt_form,tar_drift1_tt_form,tar_driftx_tt_form,tar_drift2_tt_form,tar_graph1_tt_form,tar_graph2_tt_form,tar_possession1_tt_form,tar_possession2_tt_form,tar_shont1_tt_form,tar_shont2_tt_form,tar_shofft1_tt_form,tar_shofft2_tt_form,tar_corners1_tt_form,tar_corners2_tt_form,tar_offsides1_tt_form,tar_offsides2_tt_form,tar_fouls1_tt_form,tar_fouls2_tt_form,tar_cards1_tt_form,tar_cards2_tt_form,tar_gksaves1_tt_form,tar_gksaves2_tt_form,tar_precision1_tt_form,tar_precision2_tt_form,opp_w1_tt_avg,opp_wx_tt_avg,opp_w2_tt_avg,opp_ht1_tt_avg,opp_ht2_tt_avg,opp_ft1_tt_avg,opp_ft2_tt_avg,opp_psht_tt_avg,opp_psft_tt_avg,opp_vote1_tt_avg,opp_votex_tt_avg,opp_vote2_tt_avg,opp_elo1_tt_avg,opp_elo2_tt_avg,opp_oddsprob1_tt_avg,opp_oddsprobx_tt_avg,opp_oddsprob2_tt_avg,opp_drift1_tt_avg,opp_driftx_tt_avg,opp_drift2_tt_avg,opp_graph1_tt_avg,opp_graph2_tt_avg,opp_possession1_tt_avg,opp_possession2_tt_avg,opp_shont1_tt_avg,opp_shont2_tt_avg,opp_shofft1_tt_avg,opp_shofft2_tt_avg,opp_corners1_tt_avg,opp_corners2_tt_avg,opp_offsides1_tt_avg,opp_offsides2_tt_avg,opp_fouls1_tt_avg,opp_fouls2_tt_avg,opp_cards1_tt_avg,opp_cards2_tt_avg,opp_gksaves1_tt_avg,opp_gksaves2_tt_avg,opp_precision1_tt_avg,opp_precision2_tt_avg,opp_w1_tt_form,opp_wx_tt_form,opp_w2_tt_form,opp_ht1_tt_form,opp_ht2_tt_form,opp_ft1_tt_form,opp_ft2_tt_form,opp_psht_tt_form,opp_psft_tt_form,opp_vote1_tt_form,opp_votex_tt_form,opp_vote2_tt_form,opp_elo1_tt_form,opp_elo2_tt_form,opp_oddsprob1_tt_form,opp_oddsprobx_tt_form,opp_oddsprob2_tt_form,opp_drift1_tt_form,opp_driftx_tt_form,opp_drift2_tt_form,opp_graph1_tt_form,opp_graph2_tt_form,opp_possession1_tt_form,opp_possession2_tt_form,opp_shont1_tt_form,opp_shont2_tt_form,opp_shofft1_tt_form,opp_shofft2_tt_form,opp_corners1_tt_form,opp_corners2_tt_form,opp_offsides1_tt_form,opp_offsides2_tt_form,opp_fouls1_tt_form,opp_fouls2_tt_form,opp_cards1_tt_form,opp_cards2_tt_form,opp_gksaves1_tt_form,opp_gksaves2_tt_form,opp_precision1_tt_form,opp_precision2_tt_form,tar_w1_ts_avg,tar_wx_ts_avg,tar_w2_ts_avg,tar_ht1_ts_avg,tar_ht2_ts_avg,tar_ft1_ts_avg,tar_ft2_ts_avg,tar_psht_ts_avg,tar_psft_ts_avg,tar_vote1_ts_avg,tar_votex_ts_avg,tar_vote2_ts_avg,tar_elo1_ts_avg,tar_elo2_ts_avg,tar_oddsprob1_ts_avg,tar_oddsprobx_ts_avg,tar_oddsprob2_ts_avg,tar_drift1_ts_avg,tar_driftx_ts_avg,tar_drift2_ts_avg,tar_graph1_ts_avg,tar_graph2_ts_avg,tar_possession1_ts_avg,tar_possession2_ts_avg,tar_shont1_ts_avg,tar_shont2_ts_avg,tar_shofft1_ts_avg,tar_shofft2_ts_avg,tar_corners1_ts_avg,tar_corners2_ts_avg,tar_offsides1_ts_avg,tar_offsides2_ts_avg,tar_fouls1_ts_avg,tar_fouls2_ts_avg,tar_cards1_ts_avg,tar_cards2_ts_avg,tar_gksaves1_ts_avg,tar_gksaves2_ts_avg,tar_precision1_ts_avg,tar_precision2_ts_avg,tar_w1_ts_form,tar_wx_ts_form,tar_w2_ts_form,tar_ht1_t

In [ ]:
df_teams=pd.read_csv('data/teams.csv', index_col=None)
df_teams=df_teams[['tid','op_tid']].drop_duplicates()

df=df.merge(df_teams, left_on='tid1', right_on='tid')
df=df.rename(columns={'op_tid':'op_tid1'})
df=df.drop(columns=['tid'])
df=df.merge(df_teams, left_on='tid2', right_on='tid')
df=df.rename(columns={'op_tid':'op_tid2'})
df=df.drop(columns=['tid'])

df_op=dp._provide_op()
df_op=df_op.rename(columns={'tid1':'op_tid1','tid2':'op_tid2'})
df=df.merge(df_op[['op_tid1','op_tid2', 'oddsprob_home', 'oddsprob_draw', 'oddsprob_away', 'drift_home', 'drift_away', 'drift_draw']], on=['op_tid1','op_tid2'], how='left')

In [64]:
df.merge(df_op[['op_tid1','op_tid2', 'oddsprob_home', 'oddsprob_draw', 'oddsprob_away', 'drift_home', 'drift_away', 'drift_draw']], on=['op_tid1','op_tid2'], how='left')

,awayScoreHT,country,country_id,ds,homeScoreHT,liga,mid,round,sc1,sc2,t1,t2,tid1,tid2,winner,formation_h,formation_a,home_formation,away_formation,vote_home,vote_draw,vote_away,votes,y,pop_r,de,elo1,elo2,op_tid1,op_tid2,oddsprob_home,oddsprob_draw,oddsprob_away,drift_home,drift_away,drift_draw
0,NaN,england,15,2021-01-31 12:00:00+00:00,NaN,premier-league,8897050,21,NaN,NaN,chelsea,burnley,4,3,draw,NaN,NaN,NaN,NaN,0.762910,0.160065,0.077026,27770.0,2021,4.0,2021-01-31,1828.057861,1719.795166,256,135,0.713223,0.186128,0.100649,-0.068265,0.210811,0.081265
1,NaN,england,15,2021-01-31 14:00:00+00:00,NaN,premier-league,8897051,21,NaN,NaN,leicester city,leeds united,6,796,draw,4-2-3-1,4-1-4-1,14.0,12.0,0.689995,0.206004,0.104001,23019.0,2021,3.0,2021-01-31,1821.147705,1676.518799,239,1503,0.533874,0.238475,0.227651,-0.041660,0.058843,0.025857
2,NaN,england,15,2021-01-31 16:30:00+00:00,NaN,premier-league,8897027,21,NaN,NaN,west ham united,liverpool,120,181,draw,4-2-3-1,4-3-3,14.0,18.0,0.193081,0.186881,0.620038,28066.0,2021,4.0,2021-01-31,1757.818237,1944.577148,444,1183,0.218530,0.240785,0.540685,-0.091515,0.037856,-0.022932
3,NaN,england,15,2021-01-31 19:15:00+00:00,NaN,premier-league,8897039,21,NaN,NaN,brighton hove albion,tottenham,609,274,draw,3-5-2,4-2-3-1,9.0,14.0,0.119534,0.192975,0.687491,21609.0,2021,3.0,2021-01-31,1662.046997,1836.129150,984,680,0.291167,0.286855,0.421978,-0.078422,0.078682,-0.059461
4,NaN,spain,42,2021-01-31 15:15:00+00:00,NaN,laliga,8966464,21,NaN,NaN,cadiz,atletico madrid,940,134,draw,4-4-2,4-4-2,20.0,20.0,0.126301,0.173107,0.700591,22668.0,2021,3.0,2021-01-31,1598.655518,1927.645752,1588,261,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,NaN,austria,2,2021-01-31 13:30:00+00:00,NaN,bundesliga,8858506,15,NaN,NaN,skn st polten,wolfsberger ac,1033,330,draw,NaN,NaN,NaN,NaN,0.195041,0.347658,0.457300,1815.0,2021,0.0,2021-01-31,1419.237427,1550.491699,1395,368,NaN,NaN,NaN,NaN,NaN,NaN
75,NaN,turkey,45,2021-01-31 13:00:00+00:00,NaN,tff-1-lig,8917288,19,NaN,NaN,boluspor,adana demirspor,410,153,draw,NaN,NaN,NaN,NaN,0.223048,0.267658,0.509294,807.0,2021,0.0,2021-01-31,NaN,NaN,6537,4729,NaN,NaN,NaN,NaN,NaN,NaN
76,NaN,belgium,4,2021-01-31 12:30:00+00:00,NaN,pro-league,8833161,23,NaN,NaN,club brugge,standard liege,82,270,draw,NaN,NaN,NaN,NaN,0.663458,0.254644,0.081898,7375.0,2021,1.0,2021-01-31,1688.555908,1521.353760,562,1355,NaN,NaN,NaN,NaN,NaN,NaN
77,NaN,germany,20,2021-01-31 13:00:00+00:00,NaN,3-liga,8897913,22,NaN,NaN,tsv 1860 munchen,fsv zwickau,296,1161,draw,NaN,NaN,NaN,NaN,0.693064,0.254796,0.052140,2033.0,2021,0.0,2021-01-31,NaN,NaN,1193,2159,0.505812,0.280372,0.213816,0.063097,-0.084783,-0.082277


In [75]:
df_op


,bn,country,country_id,ds,liga,mid,odds_away,odds_draw,odds_home,sc1,sc2,t1,t2,op_tid1,op_tid2,winner,oddsprob_home,oddsprob_draw,oddsprob_away,drift_home,drift_away,drift_draw
1,0.769231,spain,79,2021-01-31 16:00:00+00:00,segunda-division-b-group-4,j3Y9nKmF,3.31,3.01,2.21,0,0,ucam murcia,cordoba,1607,201,draw,0.413793,0.304438,0.281769,-0.039783,0.024380,0.023873
2,0.538462,portugal,66,2021-01-31 15:00:00+00:00,campeonato-de-portugal,0QsOhlr3,4.96,3.32,1.66,0,0,canelas 2010,sc espinho,5144,6636,draw,0.540894,0.273849,0.185257,-0.026903,0.044446,-0.016034
3,0.538462,england,25,2021-01-31 14:00:00+00:00,women-s-super-league,QVy5J7LG,32.79,15.32,1.02,0,0,manchester city w,west ham w,1188,1859,draw,0.897301,0.069221,0.033478,-0.011119,0.233352,-0.010093
4,0.692308,hungary,35,2021-01-31 12:00:00+00:00,merkantil-bank-liga,QoKnxPsq,3.54,3.38,1.93,0,0,budaorsi sc,szolnoki mav,5396,5461,draw,0.471456,0.272429,0.256115,-0.008963,0.001408,0.011767
5,0.076923,italy,41,2021-01-31 13:30:00+00:00,serie-d-group-i,6Xcr5myQ,2.67,2.98,2.50,0,0,s agata,santa maria cilento,6613,5365,draw,0.360790,0.302825,0.336386,0.000974,-0.001958,0.000974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,0.461538,world,91,2021-01-31 09:00:00+00:00,club-friendly,8rRnc5CU,2.42,3.68,2.44,0,0,gwelup croatia sc aus,ecu joondalup aus,7649,7662,draw,0.370748,0.249496,0.379756,0.007891,-0.008454,0.000853
253,0.461538,spain,79,2021-01-31 11:00:00+00:00,tercera-division-group-5,SYQqLkB7,3.71,2.90,2.03,0,0,granollers,ue figueres,4994,6560,draw,0.437974,0.314028,0.247998,-0.031605,0.110817,-0.086772
254,0.076923,italy,41,2021-01-31 13:30:00+00:00,serie-d-group-b,SjlFggSk,3.95,3.05,1.90,0,0,vis nova giussano,scanzorosciate,5245,5420,draw,0.475769,0.295257,0.228974,0.003220,0.003220,-0.007784
255,0.769231,spain,79,2021-01-31 11:00:00+00:00,segunda-division-b-group-3,4KLgJQrk,6.72,3.47,1.53,0,0,ibiza,alcoyano,3844,1396,draw,0.590045,0.263150,0.146805,-0.035511,0.091277,0.013168


In [63]:
df_op[(df_op['country']=='germany') & ~(df_op['liga'].str.contains('segunda')) & ~(df_op['liga'].str.contains('tercera'))]


,bn,country,country_id,ds,liga,mid,odds_away,odds_draw,odds_home,sc1,sc2,t1,t2,op_tid1,op_tid2,winner,oddsprob_home,oddsprob_draw,oddsprob_away,drift_home,drift_away,drift_draw
72,0.846154,germany,32,2021-01-31 14:00:00+00:00,regionalliga-west,0Onkbtzj,14.65,7.33,1.14,0,0,dortmund ii,lotte,4588,3038,draw,0.799656,0.134329,0.066015,-0.028469,0.060777,-0.073277
143,0.923077,germany,32,2021-01-31 12:00:00+00:00,3-liga,G4toVcV0,2.63,3.50,2.53,0,0,duisburg,lubeck,1056,4044,draw,0.371608,0.272519,0.355873,-0.030964,0.014786,0.008388
206,0.923077,germany,32,2021-01-31 13:00:00+00:00,3-liga,bsBOrHa0,4.47,3.37,1.84,0,0,munich 1860,zwickau,1193,2159,draw,0.505812,0.280372,0.213816,0.063097,-0.084783,-0.082277


In [34]:
df_teams

,tid,op_tid
0,1751,3706
1,1318,1700
2,519,2986
3,267,721
4,614,992
...,...,...
1336,1092,2457
1337,1263,392
1339,420,1522
1341,1161,2159


In [37]:
df=df.merge(df_teams, left_on='tid1', right_on='tid')
df=df.rename(columns={'op_tid':'op_tid1'})
df=df.drop(columns=['tid'])
df=df.merge(df_teams, left_on='tid2', right_on='tid')
df=df.rename(columns={'op_tid':'op_tid2'})
df=df.drop(columns=['tid'])
df


,awayScoreHT,country,country_id,ds,homeScoreHT,liga,mid,round,sc1,sc2,t1,t2,tid1,tid2,winner,formation_h,formation_a,home_formation,away_formation,vote_home,vote_draw,vote_away,votes,y,pop_r,de,elo1,elo2,op_tid1,op_tid1,op_tid2
0,NaN,england,15,2021-01-31 12:00:00+00:00,NaN,premier-league,8897050,21,NaN,NaN,chelsea,burnley,4,3,draw,NaN,NaN,NaN,NaN,0.762910,0.160065,0.077026,27770.0,2021,4.0,2021-01-31,1828.057861,1719.795166,256,256,135
1,NaN,england,15,2021-01-31 14:00:00+00:00,NaN,premier-league,8897051,21,NaN,NaN,leicester city,leeds united,6,796,draw,4-2-3-1,4-1-4-1,14.0,12.0,0.689995,0.206004,0.104001,23019.0,2021,3.0,2021-01-31,1821.147705,1676.518799,239,239,1503
2,NaN,england,15,2021-01-31 16:30:00+00:00,NaN,premier-league,8897027,21,NaN,NaN,west ham united,liverpool,120,181,draw,4-2-3-1,4-3-3,14.0,18.0,0.193081,0.186881,0.620038,28066.0,2021,4.0,2021-01-31,1757.818237,1944.577148,444,444,1183
3,NaN,england,15,2021-01-31 19:15:00+00:00,NaN,premier-league,8897039,21,NaN,NaN,brighton hove albion,tottenham,609,274,draw,3-5-2,4-2-3-1,9.0,14.0,0.119534,0.192975,0.687491,21609.0,2021,3.0,2021-01-31,1662.046997,1836.129150,984,984,680
4,NaN,spain,42,2021-01-31 15:15:00+00:00,NaN,laliga,8966464,21,NaN,NaN,cadiz,atletico madrid,940,134,draw,4-4-2,4-4-2,20.0,20.0,0.126301,0.173107,0.700591,22668.0,2021,3.0,2021-01-31,1598.655518,1927.645752,1588,1588,261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,NaN,austria,2,2021-01-31 13:30:00+00:00,NaN,bundesliga,8858506,15,NaN,NaN,skn st polten,wolfsberger ac,1033,330,draw,NaN,NaN,NaN,NaN,0.195041,0.347658,0.457300,1815.0,2021,0.0,2021-01-31,1419.237427,1550.491699,1395,1395,368
75,NaN,turkey,45,2021-01-31 13:00:00+00:00,NaN,tff-1-lig,8917288,19,NaN,NaN,boluspor,adana demirspor,410,153,draw,NaN,NaN,NaN,NaN,0.223048,0.267658,0.509294,807.0,2021,0.0,2021-01-31,NaN,NaN,6537,6537,4729
76,NaN,belgium,4,2021-01-31 12:30:00+00:00,NaN,pro-league,8833161,23,NaN,NaN,club brugge,standard liege,82,270,draw,NaN,NaN,NaN,NaN,0.663458,0.254644,0.081898,7375.0,2021,1.0,2021-01-31,1688.555908,1521.353760,562,562,1355
77,NaN,germany,20,2021-01-31 13:00:00+00:00,NaN,3-liga,8897913,22,NaN,NaN,tsv 1860 munchen,fsv zwickau,296,1161,draw,NaN,NaN,NaN,NaN,0.693064,0.254796,0.052140,2033.0,2021,0.0,2021-01-31,NaN,NaN,1193,1193,2159


In [27]:
df_.to_csv('data/stats_generated.csv', index=False)

In [75]:
df=pd.read_csv('data/stats_generated.csv', index_col=None)
df=df.dropna()

In [82]:
COL_CUR=['side', 'country_id', 'round', 'ds', 'de', 'form1', 'form2', 'vote1', 'votex', 'vote2', 'pop_r', 'elo1', 'elo2']
COL_PREV=['w1', 'wx', 'w2',  'ht1', 'ht2', 'ft1', 'ft2', 'ps_ht', 'ps_ft']
COL_CAT=['country_id','form1', 'form2']
COL_BIN=['side']

COL_INF=['country', 'liga', 'mid', 'round', 'ds', 't1', 't2','tid1', 'tid2', 'w1', 'wx', 'w2',  'ft1', 'ft2','winner']

In [88]:
scaler=MinMaxScaler()
nums=scaler.fit_transform(df[df.columns[25:]].values)
nums_df=pd.DataFrame(nums, columns=df.columns[25:])
df.reset_index(drop=True, inplace=True)
df=pd.concat([df[df.columns[:25]],nums_df], axis=1)

,pop_r,elo1,elo2,tar_w1_tt_avg,tar_wx_tt_avg,tar_w2_tt_avg,tar_ht1_tt_avg,tar_ht2_tt_avg,tar_ft1_tt_avg,tar_ft2_tt_avg,tar_ps_ht_tt_avg,tar_ps_ft_tt_avg,tar_vote1_tt_avg,tar_votex_tt_avg,tar_vote2_tt_avg,tar_elo1_tt_avg,tar_elo2_tt_avg,opp_w1_tt_avg,opp_wx_tt_avg,opp_w2_tt_avg,opp_ht1_tt_avg,opp_ht2_tt_avg,opp_ft1_tt_avg,opp_ft2_tt_avg,opp_ps_ht_tt_avg,opp_ps_ft_tt_avg,opp_vote1_tt_avg,opp_votex_tt_avg,opp_vote2_tt_avg,opp_elo1_tt_avg,opp_elo2_tt_avg,tar_w1_ts_avg,tar_wx_ts_avg,tar_w2_ts_avg,tar_ht1_ts_avg,tar_ht2_ts_avg,tar_ft1_ts_avg,tar_ft2_ts_avg,tar_ps_ht_ts_avg,tar_ps_ft_ts_avg,tar_vote1_ts_avg,tar_votex_ts_avg,tar_vote2_ts_avg,tar_elo1_ts_avg,tar_elo2_ts_avg,opp_w1_ts_avg,opp_wx_ts_avg,opp_w2_ts_avg,opp_ht1_ts_avg,opp_ht2_ts_avg,opp_ft1_ts_avg,opp_ft2_ts_avg,opp_ps_ht_ts_avg,opp_ps_ft_ts_avg,opp_vote1_ts_avg,opp_votex_ts_avg,opp_vote2_ts_avg,opp_elo1_ts_avg,opp_elo2_ts_avg,diff_w1_tt_avg,diff_wx_tt_avg,diff_w2_tt_avg,diff_ht1_tt_avg,diff_ht2_tt_avg,diff_ft1_tt_avg,diff_ft2_tt_avg,diff_ps_ht_tt_avg,diff_ps_ft_tt_avg,diff_vote1_tt_avg,diff_votex_tt_avg,diff_vote2_tt_avg,diff_elo1_tt_avg,diff_elo2_tt_avg,diff_w1_ts_avg,diff_wx_ts_avg,diff_w2_ts_avg,diff_ht1_ts_avg,diff_ht2_ts_avg,diff_ft1_ts_avg,diff_ft2_ts_avg,diff_ps_ht_ts_avg,diff_ps_ft_ts_avg,diff_vote1_ts_avg,diff_votex_ts_avg,diff_vote2_ts_avg,diff_elo1_ts_avg,diff_elo2_ts_avg,diff_vote12,diff_elo
0,1.00,0.472463,0.472463,1.000000,0.000000,0.000000,0.571429,0.000000,0.500000,0.000000,0.823529,0.782609,0.858299,0.162021,0.104519,1.000000,0.635697,0.708861,0.189873,0.101266,0.211573,0.041772,0.292194,0.094937,0.650782,0.624656,0.708244,0.362246,0.156602,0.836029,0.678962,1.000000,0.000000,0.000000,0.500000,0.000000,0.500000,0.000000,0.777778,0.785714,0.861679,0.162021,0.097464,0.995360,0.635697,0.000000,0.000000,1.000000,0.000000,0.400000,0.166667,0.375000,0.333333,0.428571,0.332264,0.305874,0.560808,0.892168,0.931487,0.645570,0.405063,0.449367,0.709916,0.476503,0.624684,0.442814,0.640587,0.618465,0.598509,0.322713,0.469866,0.647809,0.455834,1.000000,0.500000,0.000000,0.750000,0.300000,0.666667,0.306452,0.833333,0.750000,0.785620,0.368704,0.246279,0.596018,0.186188,0.725685,0.500000
1,0.75,0.628003,0.657831,1.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.588235,0.695652,0.085197,0.391887,0.799607,0.464370,0.471198,0.442748,0.282443,0.274809,0.196292,0.103817,0.258270,0.213740,0.607993,0.544972,0.475809,0.485643,0.336685,0.648486,0.676920,1.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.555556,0.714286,0.107016,0.391887,0.745632,0.470917,0.471198,1.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.555556,0.714286,0.122613,0.410600,0.720589,0.470917,0.471198,0.778626,0.358779,0.362595,0.385496,0.441603,0.545038,0.371251,0.483921,0.613010,0.243569,0.416984,0.767833,0.334032,0.289995,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.491586,0.482920,0.513713,0.500000,0.500000,0.626219,0.472435
2,1.00,0.811409,0.523643,1.000000,0.000000,0.000000,0.857143,0.000000,1.000000,0.166667,0.941176,0.956522,0.874874,0.211592,0.060606,0.464370,0.471198,0.408333,0.266667,0.325000,0.180952,0.120000,0.229167,0.225000,0.592157,0.523913,0.424758,0.522580,0.371003,0.468630,0.516784,1.000000,0.000000,0.000000,0.750000,0.000000,1.000000,0.125000,0.888889,0.928571,0.877859,0.211592,0.056515,0.470917,0.471198,0.000000,0.000000,1.000000,0.000000,0.600000,0.166667,0.750000,0.222222,0.214286,0.065924,0.211592,0.876629,0.470917,0.471198,0.795833,0.366667,0.337500,0.894444,0.432500,0.962500,0.464862,0.784043,0.824457,0.795494,0.224639,0.320414,0.496160,0.453465,1.000000,0.500000,0.000000,0.875000,0.200000,0.916667,0.177419,1.000000,1.000000,0.938040,0.500000,0.050917,0.500000,0.500000,0.878608,0.765927
3,1.00,0.756613,0.697733,1.000000,0.000000,0.000000,0.571429,0.000000,0.666667,0.000000,0.823529,0.869565,0.858765,0.293601,0.033987,0.464370,0.471198,0.379032,0.274194,0.346774,0.168203,0.087097,0.206989,0.181452,0.606262,

In [106]:
df_info=df[COL_INF]
labels=df[['w1', 'wx', 'w2']].values

encoder = OneHotEncoder()
countries=encoder.fit_transform(df[['country_id']]).toarray()
encoder = OneHotEncoder()
form1=encoder.fit_transform(df[['form1']]).toarray()
encoder = OneHotEncoder()
form2=encoder.fit_transform(df[['form2']]).toarray()
side=df[['side']].values

data=np.hstack([nums,countries,form1,form2,side])

In [108]:
data.shape,labels.shape

((130140, 187), (130140, 3))